In [130]:
import pandas as pd
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [112]:
df = pd.read_csv("heart.csv")
print(df.isna().any(axis=0))
df

Age               False
Sex               False
ChestPainType     False
RestingBP         False
Cholesterol       False
FastingBS         False
RestingECG        False
MaxHR             False
ExerciseAngina    False
Oldpeak           False
ST_Slope          False
HeartDisease      False
dtype: bool


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [75]:
Y = df["HeartDisease"]
a = df.select_dtypes(['object']).columns
X = df.drop(a,axis=1)
b = pd.get_dummies(df[a],drop_first=1)

In [88]:
s = StandardScaler()
c = s.fit_transform(X.drop(["FastingBS","HeartDisease"],axis=1))
k = df.select_dtypes('number')
x_scaled = pd.DataFrame(c,columns = X.drop(["FastingBS","HeartDisease"],axis = 1).columns,index = X.drop(["FastingBS","HeartDisease"],axis = 1).index)
x_scaled = x_scaled[(np.abs(c)<3).all(axis=1)]
filter = (np.abs(c)<3).all(axis=1)
final_x = pd.concat([x_scaled,X["FastingBS"][filter],b[filter]],axis=1)
final_x

,Age,RestingBP,Cholesterol,MaxHR,Oldpeak,FastingBS,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up
0,-1.433140,0.410909,0.825070,1.382928,-0.832432,0,True,True,False,False,True,False,False,False,True
1,-0.478484,1.491752,-0.171961,0.754157,0.105664,0,False,False,True,False,True,False,False,True,False
2,-1.751359,-0.129513,0.770188,-1.525138,-0.832432,0,True,True,False,False,False,True,False,False,True
3,-0.584556,0.302825,0.139040,-1.132156,0.574711,0,False,False,False,False,True,False,True,True,False
4,0.051881,0.951331,-0.034755,-0.581981,-0.832432,0,True,False,True,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,-0.902775,-1.210356,0.596393,-0.188999,0.293283,0,True,False,False,True,True,False,False,True,False
914,1.536902,0.627078,-0.053049,0.164684,2.357094,1,True,False,False,False,True,False,False,True,False
915,0.370100,-0.129513,-0.620168,-0.857069,0.293283,0,True,False,False,False,True,False,True,True,False
916,0.370100,-0.129513,0.340275,1.461525,-0.832432,0,False,True,False,False,False,False,False,True,False


In [121]:
trainX,testX,trainY,testY = train_test_split(final_x,Y[filter],stratify=Y[filter],train_size=0.8,random_state=1)  
#Stratify preserves the ratio of 1's and 0's in training set and testing set in targets

In [159]:
model1 = cross_val_score(SVC(),final_x,Y[filter],cv=5)
print(f"Support Vector without bagging with cross validation {model1.mean()}") 
print(f"Support Vector Machine with bagging with cross validation {cross_val_score(bag1,final_x,Y[filter],cv=5).mean()}")
print(f"Support Vector Machine without bagging without cross validation {(SVC().fit(trainX,trainY)).score(testX,testY)}")
bag1 = BaggingClassifier(SVC(),n_estimators = 10,max_samples=0.8,oob_score=False,random_state=1)
bag1.fit(trainX,trainY)
print(f"Support Vector Machine with bagging without cross validation {bag1.score(testX,testY)}")

Support Vector without bagging with cross validation 0.8296958410924893
Support Vector Machine with bagging with cross validation 0.8297144630664185
Support Vector Machine without bagging without cross validation 0.8777777777777778
Support Vector Machine with bagging without cross validation 0.8777777777777778


In [ ]:
bag2 = BaggingClassifier(SVC(),n_estimators = 10,max_samples=0.8,oob_score=True,random_state=2)
bag2.fit(trainX,trainY)
bag2.oob_score_
#We can see this score is much higher because there are not much out of the bag(oob samples) which are not included in any of the bootstrap samples.

c:\Users\user\Desktop\ML Project\ml\lib\site-packages\sklearn\ensemble\_bagging.py:875: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
c:\Users\user\Desktop\ML Project\ml\lib\site-packages\sklearn\ensemble\_bagging.py:881: RuntimeWarning: invalid value encountered in divide
  oob_decision_function = predictions / predictions.sum(axis=1)[:, np.newaxis]


0.8650904033379694

In [161]:
model3 = cross_val_score(DecisionTreeClassifier(),final_x,Y[filter],cv=5)
bag3 = BaggingClassifier(DecisionTreeClassifier(),n_estimators = 10,max_samples=0.8,oob_score=False,random_state=1)
bag3.fit(trainX,trainY)
print(f"Decision Tree without bagging {model3.mean()}") 
print(f"Decision Tree Machine with bagging {cross_val_score(bag3,final_x,Y[filter],cv=5).mean()}")
print(f"Decision Tree Machine without bagging without cross validation {(DecisionTreeClassifier().fit(trainX,trainY)).score(testX,testY)}")
print(f"Decision Tree Machine with bagging without cross validation {bag3.score(testX,testY)}")

Decision Tree without bagging 0.7251272501551831
Decision Tree Machine with bagging 0.7996896337678461
Decision Tree Machine without bagging without cross validation 0.8388888888888889
Decision Tree Machine with bagging without cross validation 0.8277777777777777


In [163]:
bag3 = BaggingClassifier(DecisionTreeClassifier(),n_estimators = 10,max_samples=0.8,oob_score=True,random_state=1)
bag3.fit(trainX,trainY)
bag3.oob_score_

c:\Users\user\Desktop\ML Project\ml\lib\site-packages\sklearn\ensemble\_bagging.py:875: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
c:\Users\user\Desktop\ML Project\ml\lib\site-packages\sklearn\ensemble\_bagging.py:881: RuntimeWarning: invalid value encountered in divide
  oob_decision_function = predictions / predictions.sum(axis=1)[:, np.newaxis]


0.8108484005563282